In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 119.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00


In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Hawk28/CodeLlama-7B-RP")
model = AutoModelForCausalLM.from_pretrained("Hawk28/CodeLlama-7B-RP", load_in_8bit=True)

(…)7B-RP/resolve/main/tokenizer_config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

(…)eLlama-7B-RP/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-RP/resolve/main/special_tokens_map.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

(…)CodeLlama-7B-RP/resolve/main/config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)B-RP/resolve/main/generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and genres of all the books?
#
### SQL:
#
# SELECT title ,  genre FROM books
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( Customer_ID, Customer_Name, Customer_Address, Customer_Phone, Customer_Email, Customer_Status )
# Products ( Product_ID, Product_Name, Product_Price, Product_Quantity, Product_Status )
# Orders ( Order_ID, Customer_ID,


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the number of books published in each genre
#
### SQL:
#
# SELECT count(*) ,  genre FROM books GROUP BY genre
# 
### End.

### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Find the most borrowed book
#
### SQL:
#
# SELECT T1.title FROM books AS T1 JOIN borrowedbooks AS T2 ON T1.book_id  =  T2.book_id GROUP BY T1.title ORDER BY count(*) DESC LIMIT 1
# 
### End.

### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the IDs, titles, and publication years of "Fantasy" genre books?
#
### SQL:
#
# SELECT book_id ,  title ,  publication_year FROM books WHERE genre  =  "Fantasy"
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( customer_id, customer_name, customer_address, customer_phone, customer_email, customer_city, customer_state, customer_zip_code, customer_country )
# Products ( product_id


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of book titles and their respective authors?
#
### SQL:
#
# SELECT t1.title ,  t2.author_name FROM books AS t1 JOIN authors AS t2 ON t1.author_id  =  t2.author_id
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( customer_id, customer_name, customer_address, customer_phone, customer_email, customer_city, customer_state, customer_country


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Which members borrowed books along with the borrow and return dates?
#
### SQL:
#
# SELECT T1.member_name ,  T2.book_id ,  T2.borrow_date ,  T2.return_date FROM members AS T1 JOIN borrowedbooks AS T2 ON T1.member_id  =  T2.member_id
# 
### End.

### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What are the titles and authors of books by authors born after 1950?
#
### SQL:
#
# SELECT title ,  author_name FROM books AS T1 JOIN authors AS T2 ON T1.author_id  =  T2.author_id WHERE T2.birth_year  >  1950
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( customer_id, customer_name, customer_address, customer_phone, customer_email, customer_


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Could you provide a list of unique genres available in the collection?
#
### SQL:
# 
# SELECT DISTINCT genre FROM books
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( Customer_ID, Customer_Name, Customer_Address, Customer_Phone, Customer_Email, Customer_Status )
# Products ( Product_ID, Product_Name, Product_Price, Product_Quantity, Product_Status )
# Orders ( Order_ID, Customer_ID


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Show the members who joined before 1st January 2022 (Format: DD-MM-YYYY) and the books they borrowed:
#
### SQL:
#
# SELECT T1.member_name ,  T2.title FROM Members AS T1 JOIN BorrowedBooks AS T2 ON T1.member_id  =  T2.member_id WHERE T1.join_date  <  "2022-01-01";
# 
### End.

### SQL tables followed by foreign key information:
#
# Books ( book_id, title


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# Can you provide a list of members along with the number of books they have borrowed, sorted by the number of borrows in descending order?
#
### SQL:
#
# SELECT T1.member_name ,  count(*) FROM borrowedbooks AS T1 JOIN members AS T2 ON T1.member_id  =  T2.member_id GROUP BY T1.member_id ORDER BY count(*) DESC
# 
### End.

### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Auth


In [ ]:
batch = tokenizer("""### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
""", return_tensors='pt')

with torch.cuda.amp.autocast():
    input_ids = batch['input_ids'].to('cuda')  # Move input_ids to the CUDA device
    output_tokens = model.generate(input_ids=input_ids, max_new_tokens=100)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




 ### Complete SQL query only and with no explanation
### SQL tables followed by foreign key information:
#
# Books ( book_id, title, author_id, genre, publication_year )
# Authors ( author_id, author_name, birth_year )
# Members ( member_id, member_name, join_date )
# BorrowedBooks ( borrow_id, book_id, member_id, borrow_date, return_date )
#
# BorrowedBooks.book_id can be joined with Books.book_id
# BorrowedBooks.member_id can be joined with Members.member_id
# Authors.author_id can be joined with Books.author_id
#
### Question:
#
# What is the earliest publication year of a book in the "Fiction" genre?
#
### SQL:
# 
# SELECT publication_year FROM books WHERE genre  =  "Fiction" ORDER BY publication_year ASC LIMIT 1
# 
### End.

### SQL tables followed by foreign key information:
#
# Customers ( customer_id, customer_name )
# Products ( product_id, product_name )
# Orders ( order_id, customer_id, order_date )
# OrderItems ( order_
